In [7]:
import requests

PREFIX = 'http://127.0.0.1:8889'

# Define a schema and the keys

The schema has two roles. Firstly, it defines the format of the data, such as the type of each column, minimum and maximum lengths, and permitted values. Secondly, it contains all the hashing settings, from the hash size, through global salts, to the tokenisation settings of each field.

The schema is designed to be portable between Clkhash and Anonlink. Two runs of Clkhash with the same schema and the same keys on the same data will produce the exact same hashes, even if they use different versions of Clkhash.

The keys can be thought of as a secret salt. They must be shared between organisations that are hashing their data, but must not be given to the authority performing the linkage.

In [13]:
SCHEMA = {
  'version': 1,
  'clkConfig': {
    'l': 1024,
    'k': 20,
    'hash': {
      'type': 'doubleHash'
    },
    'kdf': {
      'type': 'HKDF',
      'hash': 'SHA256',
      'salt': 'SCbL2zHNnmsckfzchsNkZY9XoHk96P/G5nUBrM7ybymlEFsMV6PAeDZCNp3rfNUPCtLDMOGQHG4pCQpfhiHCyA==',
      'info': 'c2NoZW1hX2V4YW1wbGU=',
      'keySize': 64
    }
  },
  'features': [
    {
      'identifier': 'NAME freetext',
      'format': {
        'type': 'string',
        'encoding': 'utf-8',
        'case': 'mixed',
        'minLength': 3
      },
      'hashing': {
        'ngram': 2,
        'weight': 0.5
      }
    },
    {
      'identifier': 'DOB YYYY/MM/DD',
      'format': {
        'type': 'string',
        'encoding': 'ascii',
        'description': 'Numbers separated by slashes, in the year, month, day order',
        'pattern': '\\d\\d\\d\\d/\\d\\d/\\d\\d'
      },
      'hashing': {
        'ngram': 1,
        'positional': True
      }
    },
    {
      'identifier': 'GENDER M or F',
      'format': {
        'type': 'enum',
        'values': ['M', 'F']
      },
      'hashing': {
        'ngram': 1,
        'weight': 2
      }
    }
  ]
}

In [14]:
KEY1, KEY2 = 'correct', 'horse'

# Make new project

In [23]:
r = requests.post(
    PREFIX + '/projects/{project_id}'.format(
        project_id='demo-data'),
    params=dict(
        key1=KEY1,
        key2=KEY2),
    json=SCHEMA)

print('Status code:', r.status_code)

Status code: 201


# See all projects

In [21]:
r = requests.get(
    PREFIX + '/projects')

print('Status code:', r.status_code)
print('Data:', r.json())

Status code: 200
Data: {'projects': ['bar', 'demo-data']}


# Upload private data

In [33]:
csv_data = (
    'NAME freetext,DOB YYYY/MM/DD,GENDER M or F\n'
    'Jane Doe,1968/05/19,F\n'
    'Peter Griffin,1998/12/20,M\n')

r = requests.post(
    PREFIX + '/projects/{project_id}/clks/'.format(
        project_id='demo-data'),
    params=dict(
        header=True),
    data=csv_data)

json = r.json()
clk_start_index = json['clk_start_index']
clk_number = json['clk_number']

print('Status code:', r.status_code)
print('Data:', r.json())

Status code: 200
Data: {'clk_number': 2, 'clk_start_index': 6}


# Get clks

In [41]:
r = requests.get(
    PREFIX + '/projects/{project_id}/clks/{offset}/{limit}'.format(
        project_id='demo-data',
        offset=clk_start_index,
        limit=clk_number))

print('Status code:', r.status_code)
print('Data:', r.json())

Status code: 200
Data: {'clks': [{'errMsg': None, 'hash': 'EgBsCgAQACwDiJiri8oACaApBgB2UigAYJAAADCECKjBgABKJFFNpBVGzMREZBGBmCIwAODcgqGI9hAFKAkndGAhAGClqQEEAiLue2siEJQgkUEggGBAAQgGAWAALZACtSAiABMMQlihFVU1MTHEAmlACIMGiIwCsCAoEEDQAAA=', 'index': 6, 'status': 'done'}, {'errMsg': None, 'hash': '2wRKwCio1SQDeAAEowBECdwNGkAJgChviDATAHTDRCgQECHqxiEARgl+iLQADIspmCB7gcFUgKGIwvCMBAirULh5kkDaiTlAJowckX8A0BEgk8MgkABIF2EmByhJK6AiMwCGjlGYIlCCwiQAICED4QEgBAMsIBiAMBDkGyCSQAI=', 'index': 7, 'status': 'done'}]}


# Upload lots of private data!

In [48]:
with open('fake-data.csv') as f:
    csv_data = f.read()

r = requests.post(
    PREFIX + '/projects/{project_id}/clks/'.format(
        project_id='demo-data'),
    params=dict(
        header=True),
    data=csv_data)

json = r.json()
clk_start_index = json['clk_start_index']
clk_number = json['clk_number']

print('Status code:', r.status_code)
print('Data:', r.json())

Status code: 200
Data: {'clk_number': 100000, 'clk_start_index': 200006}


# Check job status

In [50]:
r = requests.get(
    PREFIX + '/projects/{project_id}/clks'.format(
        project_id='demo-data'))

print('Status code:', r.status_code)
print('Data:', r.json())

Status code: 200
Data: {'clks_status': [{'count': 4, 'index': 2, 'status': 'done'}, {'count': 192000, 'index': 100006, 'status': 'done'}, {'count': 8000, 'index': 292006, 'status': 'queued'}]}


# Delete clks or abort jobs

If a particular clk has already been computed, then we simply delete it from the database. Otherwise, we cancel the scheduled job and delete the private data.

In [42]:
r = requests.delete(
    PREFIX + '/projects/{project_id}/clks/{offset}/{limit}'.format(
        project_id='demo-data',
        offset=clk_start_index,
        limit=clk_number))

print('Status code:', r.status_code)

Status code: 204


# Delete project

This will also delete all clks associated with the project and abort all hashing jobs.

In [22]:
r = requests.delete(
    PREFIX + '/projects/{project_id}'.format(
        project_id='demo-data'))

print('Status code:', r.status_code)

Status code: 204
